In [48]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from matplotlib.patches import Circle


In [49]:

# JSON 파일 로드
with open(r"C:\Users\hi020\Desktop\play\ac\annotations.json") as f:
    data = json.load(f)

data_dict ={}

num=400
for num in range(600,799):
    bbox_list = []

    # 바운딩 박스 정보 수집 함수 정의
    def yxwh(x, bbox_list):
        if x:
            if type(x) == list:
                for _x in x:
                    if type(_x) == dict:
                        a = _x.keys()
                        for i in a:
                            if i == 'bboxes':
                                bbox_list += _x[i]
                            elif i == 'bbox':
                                bbox_list.append(_x[i])
                            else:
                                yxwh(_x[i], bbox_list)
            elif type(x) == dict:
                a = x.keys()
                for i in a:
                    if i == 'bboxes':
                        bbox_list += x[i]
                    elif i == 'bbox':
                        if list(x[i].keys()) == ['bbox']:
                            bbox_list.append(x[i]['bbox'])
                        else:
                            bbox_list.append(x[i])
                    else:
                        yxwh(x[i], bbox_list)
                    
    yxwh(data[num]['models'], bbox_list)

    points = []
    points.append((data[num]['general_figure_info']['plot_info']['bbox']['x'] + data[num]['general_figure_info']['plot_info']['bbox']['w']/2,
                data[num]['general_figure_info']['plot_info']['bbox']['y'] + data[num]['general_figure_info']['plot_info']['bbox']['h']/2))

    points.append((points[-1][0] + data[num]['general_figure_info']['plot_info']['bbox']['w']/2,
                points[-1][1]))

    radian = 0.
    total_span = sum([i['span'] for i in data[num]['models']])  # 전체 span 합산
    for i in data[num]['models']:
        radian += i['span']
        points.append((points[0][0] + data[num]['general_figure_info']['plot_info']['bbox']['w']/2*np.cos(radian),
                    points[0][1] - data[num]['general_figure_info']['plot_info']['bbox']['w']/2*np.sin(radian)))


    patches = []

    for p1 in points:
        patches.append(Circle(p1, radius=2, color='red'))

    fig, ax = plt.subplots(1, figsize=(10,10))
    ax.set_axis_off()
    for p in patches:
        ax.add_patch(p)
    # 범례 추가
    legend_item = data[num]['general_figure_info']['legend']['items']
    legend_num = len(legend_item)
    legend_labels = []

    for k in range(legend_num):
        model_name = legend_item[k]['model']
        # 각 모델의 비율 계산
        model_span = data[num]['models'][k]['span']
        model_percentage = (model_span / total_span) * 100  # 비율을 퍼센테이지로 변환
        legend_label = "{}: {:.1f}%".format(model_name, model_percentage)  # 모델명과 비율을 포함한 레이블
        ax.text((points[k+1][0]+points[k+2][0])/2, (points[k+1][1]+points[k+2][1])/2, legend_label, ha='center', va='center', fontsize=12, color='black')

    # 범례를 그래프 내부에 추가
    #ax.imshow(plt.imread(img_lo))

    fig_type = data[num].get('type', 'unknown')

    center = points[0]
    i = len(points)-1
    edge=[]

    for k in range(1, len(points)-1):
        edge.append(points[k])
    
    # data_dict에 저장
    data_dict[num] = {'type':fig_type, 'center': center, 'edge': edge}

# 데이터 출력
for num, data in data_dict.items():
    print(f"num: {num}, data: {data}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hi020\\Desktop\\play\\ac\\annotations.json'

In [ ]:
import json
import pickle
import numpy as np

# JSON 파일 로드
with open(r"C:\Users\hi020\Desktop\play\ac\annotations.json") as f:
    data = json.load(f)

data_dict = {}

# 바운딩 박스 정보 수집 함수 정의
def yxwh(x, bbox_list):
    if isinstance(x, list):
        for _x in x:
            if isinstance(_x, dict):
                for key, value in _x.items():
                    if key == 'bboxes':
                        bbox_list.extend(value)
                    elif key == 'bbox':
                        bbox_list.append(value)
                    else:
                        yxwh(value, bbox_list)
    elif isinstance(x, dict):
        for key, value in x.items():
            if key == 'bboxes':
                bbox_list.extend(value)
            elif key == 'bbox':
                bbox_list.append(value)
            else:
                yxwh(value, bbox_list)

# 주어진 범위의 데이터 처리
for num in range(600, 799):
    bbox_list = []
    yxwh(data[num]['models'], bbox_list)

    plot_info = data[num]['general_figure_info']['plot_info']
    plot_bbox = plot_info['bbox']
    
    points = [
        (plot_bbox['x'] + plot_bbox['w'] / 2, plot_bbox['y'] + plot_bbox['h'] / 2),
        (plot_bbox['x'] + plot_bbox['w'], plot_bbox['y'] + plot_bbox['h'] / 2)
    ]

    radian = 0.0
    total_span = sum(model['span'] for model in data[num]['models'])

    for model in data[num]['models']:
        radian += model['span']
        points.append((
            points[0][0] + plot_bbox['w'] / 2 * np.cos(radian),
            points[0][1] - plot_bbox['w'] / 2 * np.sin(radian)
        ))

    fig_type = data[num].get('type', 'unknown')
    center = points[0]
    edges = points[1:-1]

    data_dict[num] = {'type': fig_type, 'center': center, 'edges': edges}

# 피클 파일로 저장할 경로 설정
pickle_file_path = r"C:\Users\hi020\Desktop\play\ac\data_dict.pkl"

# 데이터 피클 파일로 저장
with open(pickle_file_path, 'wb') as f:
    pickle.dump(data_dict, f)

# 데이터 피클 파일에서 로드하여 확인
with open(pickle_file_path, 'rb') as f:
    loaded_data_dict = pickle.load(f)

# 데이터 출력
for num, data in loaded_data_dict.items():
    print(f"num: {num}, data: {data}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hi020\\Desktop\\play\\ac\\annotations.json'

In [ ]:
# 데이터 피클 파일에서 로드하여 확인
with open(pickle_file_path, 'rb') as f:
    loaded_data_dict = pickle.load(f)

In [ ]:
output_file = r"C:\Users\hi020\Desktop\play\sample_train1\data_output.json"
with open(output_file, 'w') as f:
    json.dump(data_dict, f, indent=4)

print(f"Data has been successfully saved to {output_file}")

Data has been successfully saved to C:\Users\hi020\Desktop\play\sample_train1\data_output.json


In [ ]:
pickle_file_path

'C:\\Users\\hi020\\Desktop\\play\\sample_train1\\figure_data_dict.pkl'

In [ ]:
# 데이터 피클 파일에서 로드하여 확인
with open(pickle_file_path, 'rb') as f:
    loaded_data_dict = pickle.load(f)

In [ ]:
#재확인작업

In [ ]:
loaded_data_dict

{600: {'type': 'pie',
  'center': (596.2740038886353, 670.8780258350654),
  'edge': [(1000.0, 670.8780258350654),
   (948.9883677070376, 476.07699755276565),
   (366.38915519423506, 1000.0),
   (1000.0, 670.8780258350654)]},
 601: {'type': 'pie',
  'center': (588.2352941176471, 591.2034088373568),
  'edge': [(1000.0, 591.2034088373568),
   (936.3143901465312, 372.7146840858988),
   (345.510366017844, 260.83991228584506),
   (224.9466856200933, 783.7233963163304),
   (600.3505050761378, 1000.0),
   (1000.0, 591.2034088373574)]},
 602: {'type': 'pie',
  'center': (675.2706078268109, 999.9999999999999),
  'edge': [(1000.0, 999.9999999999999),
   (351.3320574868587, 965.5988015708398),
   (1000.0, 1000.0)]},
 603: {'type': 'pie',
  'center': (595.2380952380952, 647.0057846827885),
  'edge': [(1000.0, 647.0057846827885),
   (916.8743941197113, 395.61850451489863),
   (452.9537893715428, 259.349672516945),
   (333.5316257083652, 331.1197246841616),
   (203.41689175496643, 543.1375001235823),

In [ ]:
with open(r"C:\Users\hi020\Desktop\play\sample_train1\annotations.json") as f:
    data = json.load(f)
num =2
data[num]['general_figure_info']['x_axis']['major_ticks']['bboxes']

[{'y': 268, 'x': 134, 'w': 8, 'h': 8},
 {'y': 268, 'x': 296, 'w': 8, 'h': 8},
 {'y': 268, 'x': 458, 'w': 8, 'h': 8},
 {'y': 268, 'x': 134, 'w': 8, 'h': 8},
 {'y': 268, 'x': 296, 'w': 8, 'h': 8},
 {'y': 268, 'x': 458, 'w': 8, 'h': 8}]

In [ ]:
a = data[num]['general_figure_info']['y_axis']['major_ticks']['bboxes']